<a href="https://colab.research.google.com/github/lehuong240823/rag-company-knowledge-consultant-chatbot/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Started

In [119]:
# @title Install Libraries
!pip -q install pinecone llama-cpp-python streamlit
!pip -q install langchain_community langchain_openai langchain_pinecone langchain-huggingface langsmith
!npm -q install localtunnel streamlit

⠙⠹⠸⠼⠴⠦⠧npm error code E404
⠧npm error 404 Not Found - GET https://registry.npmjs.org/streamlit - Not found
⠧npm error 404
⠧npm error 404  'streamlit@*' is not in this registry.
⠧npm error 404
⠧npm error 404 Note that you can also install from a
⠧npm error 404 tarball, folder, http url, or git url.
⠧npm error A complete log of this run can be found in: /root/.npm/_logs/2025-09-25T05_36_49_095Z-debug-0.log
⠧

In [4]:
# @title Import Libraries
import os, yaml, inspect, json, datetime, pytz, hashlib
import pandas as pd
from os.path import join
from google.colab import userdata
import streamlit as st
from IPython.display import display, HTML
from langchain.chains import ConversationalRetrievalChain
from langchain_community.chat_models import ChatLlamaCpp
from langchain.document_loaders import TextLoader, CSVLoader, GithubFileLoader
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain.llms import LlamaCpp
from langchain.embeddings import LlamaCppEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_openai import ChatOpenAI
from langchain_pinecone.vectorstores import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from langsmith import Client, traceable

In [5]:
# @title Configuration
config = yaml.safe_load('''
repo: lehuong240823/rag-company-knowledge-consultant-chatbot
branch: main
ref_data: reference_data
subdomain: apchatbot
format:
  - .txt
  - .csv
chat_model:
  name: Qwen3-0.6B-Q8_0-GGUF
  path: /content/Qwen3-0.6B-Q8_0.gguf
  alter_path: /kaggle/working/Qwen3-0.6B-Q8_0.gguf
  source: https://huggingface.co/Qwen/Qwen3-0.6B-GGUF/resolve/main/Qwen3-0.6B-Q8_0.gguf
  params:
    temperature: 0
    n_ctx: 1024
    n_batch: 256
    suffix: Quý khách có cần hỗ trợ thêm gì không?
    max_tokens: 100
    top_p: 0.8
    top_k: 20
    repeat_penalty: 1.5
embedding_model:
  name: Qwen3-Embedding-0.6B-Q8_0-GGUF
  path: /content/Qwen3-Embedding-0.6B-Q8_0.gguf
  alter_path: /kaggle/working/Qwen3-Embedding-0.6B-Q8_0.gguf
  source: https://huggingface.co/Qwen/Qwen3-Embedding-0.6B-GGUF/resolve/main/Qwen3-Embedding-0.6B-Q8_0.gguf
  params:
    n_ctx: 1024
    n_batch: 256
grader_model:
  name: Llama-3.2-1B-Instruct-Q8_0-GGUF
  path: /content/llama-3.2-1b-instruct-q8_0.gguf
  alter_path: /kaggle/working/llama-3.2-1b-instruct-q8_0.gguf
  source: https://huggingface.co/hugging-quants/Llama-3.2-1B-Instruct-Q8_0-GGUF/resolve/main/llama-3.2-1b-instruct-q8_0.gguf
  params:
    temperature: 0
    n_ctx: 2048
    n_batch: 256
    max_tokens: 100
    top_p: 0.8
    top_k: 20
    repeat_penalty: 1.5
''')

In [6]:
# @title Download Quantized Model
!wget -N -q {config['chat_model']['source']}
!wget -N -q {config['embedding_model']['source']}
!wget -N -q {config['grader_model']['source']}

In [8]:
# @title Set Environment Variables
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_PROJECT'] = userdata.get('PINECONE_INDEX_NAME')
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['OPENAI_API_BASE'] = 'https://router.huggingface.co/v1' #'https://openrouter.ai/api/v1'
os.environ['OPENAI_API_KEY'] = userdata.get('HF_TOKEN') #userdata.get('OPENROUTER_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
os.environ['PINECONE_INDEX_NAME'] = userdata.get('PINECONE_INDEX_NAME')
os.environ['GITHUB_PERSONAL_ACCESS_TOKEN'] = userdata.get('GITHUB_PERSONAL_ACCESS_TOKEN')
root_path =  userdata.get('ROOT_PATH')
embedding_path = join(root_path, 'embedding_data')
reference_path = join(root_path, 'reference_data')
evaluators_path = join(root_path, 'evaluators')

# Method

###Utility Functions

In [9]:
# @title Timestamp and Hashing Utilities
def get_timestamp():
  vietnam_timezone = pytz.timezone('Asia/Ho_Chi_Minh')
  vietnam_time = datetime.datetime.now(vietnam_timezone)
  return vietnam_time.isoformat()

def md5(text):
  '''MD% hash for track change'''
  return hashlib.md5(text.encode('utf-8')).hexdigest()

In [10]:
# @title Pinecone Utilities
def get_index(index_name=os.environ['PINECONE_INDEX_NAME']):
  pc = Pinecone()
  if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

  return pc.Index(index_name)

In [11]:
# @title Document Loading and Processing Utilities
def create_text_splitter(chunk_size=100, chunk_overlap=0):
  return RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

def Loader(file=None, mode='github', repo=config['repo'], branch=config['branch'], dest_dir='reference_data', format=('.txt', '.csv')):
  if file is None and mode=='github':
    loader = GithubFileLoader(
      repo=repo,
      branch=branch,
      file_filter=lambda file_path: (file_path.endswith(format) and file_path.startswith(dest_dir))
    )

  if file is not None and mode=='local':
    if file.endswith('.csv'):
      loader = CSVLoader(file)
    elif file.endswith('.txt'):
      loader = TextLoader(file)

  return loader

def add_texts(text):
  texts = text_splitter.split_text(text)
  vectorstore.add_texts(texts)
  print(f'Added {len(texts)} text chunks to vector store.')

def add_documents(path=None, ids=None, mode='github'):
  loader = Loader(file=path, mode=mode)
  documents = loader.load()
  src = '\n'.join([doc.metadata['path'] for doc in documents]) if path is None else path

  print(f'Loaded {len(documents)} document(s) from: {src}')
  docs = text_splitter.split_documents(documents)

  for idx, each in enumerate(docs):
    part = 0 if idx==0 or each.metadata['source']!=docs[idx-1].metadata['source'] else part + 1
    each.metadata['timestamp'] = get_timestamp()
    each.metadata['md5'] = md5(each.page_content)
    if ids is None:
      id = each.metadata['path'].split('/')[-1].split('.')[0] + f'_{part}'
    else:
      id = f'{ids}_{str(idx)}'
    vectorstore.add_documents([each], ids=[id])
  #vectorstore.add_documents(documents=docs)
  print(f'Added {len(docs)} text chunk(s) to vector store.')

In [12]:
# @title LangSmith Dataset Utilities
def create_dataset(file_name=None, dataset_name=None, provider='langsmith', mode='local', overwrite=False):
  '''Load json file only'''
  if mode=='local' and file_name!=None:
    dataset_name = file_name[:-5] if dataset_name is None else dataset_name
    path = join(embedding_path, provider, file_name)
    with open(path) as f:
      if not client.has_dataset(dataset_name=dataset_name):
        client.create_dataset(dataset_name=dataset_name)
        client.create_examples(
          dataset_name=dataset_name,
          examples=json.load(f)
        )
        print(f'Create {dataset_name} successfully from {path}!')
      else:
        print(f'Dataset {dataset_name} already exists.')

  if mode=='github':
    json_loader = Loader(mode='github', dest_dir=f'embedding_data/{provider}', format=('json'))
    jsons = json_loader.load()
    for each in jsons:
      dataset_name = each.metadata['path'].split('/')[-1].split('.')[0]
      if overwrite:
        client.delete_dataset(dataset_name=dataset_name)
      if not client.has_dataset(dataset_name=dataset_name):
        client.create_dataset(dataset_name=dataset_name)
        client.create_examples(
          dataset_name=dataset_name,
          examples=json.loads(each.page_content)
        )
        print(f'Create {dataset_name} successfully from {each.metadata["source"]}!')
      else:
        print(f'Dataset {dataset_name} already exists.')


In [13]:
# @title Evaluator Utilities
def get_evaluator_path(evaluator_name, mode='github'):
  if mode=='local':
    path = join(evaluators_path, evaluator_name)
  if mode=='github':
    path = join('evaluators', evaluator_name)
  return path

def load_chat_prompt(evaluator, mode='github'):
  role_map = {'system': SystemMessagePromptTemplate, 'user': HumanMessagePromptTemplate, 'ai': AIMessagePromptTemplate}
  messages = []
  file_path = join(get_evaluator_path(evaluator, mode), 'prompt.yaml')
  if mode=='local':
    with open(file_path, 'r') as f:
      config = yaml.safe_load(f)
  if mode=='github':
    yaml_loader = Loader(mode=mode, dest_dir=file_path, format=('.yaml')).load()
    config = yaml.safe_load(yaml_loader[0].page_content)

  for msg_config in config['messages']:
    messages.append(role_map[msg_config['role']].from_template(msg_config['template']))

  return ChatPromptTemplate.from_messages(messages)

def load_output_schema(evaluator, mode='github'):
  file_path = join(get_evaluator_path(evaluator, mode), 'output_schema.yaml')
  if mode=='local':
    with open(file_path, 'r') as f:
      config = yaml.safe_load(f)
  if mode=='github':
    yaml_loader = Loader(mode=mode, dest_dir=file_path, format=('.yaml')).load()
    config = yaml.safe_load(yaml_loader[0].page_content)
  return config

def create_structured_grader_llm(evaluator=None):
  return ChatLlamaCpp(
    model_path=config['grader_model']['path'] if os.path.exists(config['grader_model']['path']) else config['grader_model']['alter_path'],
    temperature=config['grader_model']['params']['temperature'],
    n_ctx=config['grader_model']['params']['n_ctx'],
    n_batch=config['grader_model']['params']['n_batch'],
    max_tokens=config['grader_model']['params']['max_tokens'],
    top_p=config['grader_model']['params']['top_p'],
    top_k=config['grader_model']['params']['top_k'],
    repeat_penalty=config['grader_model']['params']['repeat_penalty'],
    stop=['\n'],
    grammar='''
    start ::= object

object ::= "{" ws
              "\"relevant\"" ws ":" ws boolean
              ("," ws "\"explanation\"" ws ":" ws string)?
           ws "}"

boolean ::= "true" | "false"

string ::= ESCAPED_STRING

# Standard JSON string (handles quotes inside correctly)
ESCAPED_STRING ::= < "\"" ( [^"\\] | "\\" ["\\/bfnrt] | "\\u" [0-9a-fA-F]{4} )* "\"" >

ws ::= < [ \t\n\r]* >

    ''',
    streaming=False,
    verbose=False,
  ).with_structured_output(
    load_output_schema(evaluator)
  )

In [14]:
# @title Tuning Ultilities
def chunk_size_tunning(start=50, end=100, step=10, chunk_overlap=0, k=2, sync_embedded_data=False, mode='mini'):
  for chunk_size in range(start, end+step, step):
    namespace = f'chunk{chunk_size}_overlap{chunk_overlap}'
    prefix = f'tune_{namespace}_k{k}'
    print(f'Begin chunk tunning {prefix}')

    init_global_var(namespace=namespace, chunk_size=chunk_size, chunk_overlap=chunk_overlap, k=k)

    if sync_embedded_data:
      add_documents(mode='github')

    experiment_results = experiment(chunk_size=chunk_size, overlap=chunk_overlap, k=k, experiment_prefix=prefix, mode=mode)
    pd_resutlts = experiment_results.to_pandas()
    pd_resutlts['experiment'] = prefix
    pd_resutlts['chunk'] = chunk_size
    pd_resutlts['overlap'] = chunk_overlap
    pd_resutlts['k'] = k
    print(f'Complete chunk tunning {prefix}. TOTAL: {(chunk_size-start+step)/(end-start)*100}%')

def overlap_tunning(best_chunk_size, start=10, end=30, step=10, k=2, sync_embedded_data=False, mode='mini'):
  for overlap_ratio in range(start, end+step, step):
    chunk_overlap = round(best_chunk_size*overlap_ratio/100)
    namespace = f'chunk{best_chunk_size}_overlap{chunk_overlap}'
    prefix = f'tune_{namespace}_k{k}'
    print(f'Begin overlap tunning {prefix}')

    init_global_var(namespace=namespace, chunk_size=best_chunk_size, chunk_overlap=chunk_overlap, k=k)

    if sync_embedded_data:
      add_documents(mode='github')

    experiment_results = experiment(chunk_size=best_chunk_size, overlap=chunk_overlap, k=k, experiment_prefix=prefix, mode=mode)
    pd_resutlts = experiment_results.to_pandas()
    pd_resutlts['experiment'] = prefix
    pd_resutlts['chunk'] = best_chunk_size
    pd_resutlts['overlap'] = chunk_overlap
    pd_resutlts['k'] = k
    print(f'Complete overlap tunning {prefix}. TOTAL: {(overlap_ratio-start+step)/(end-start)*100}%')


def k_tunning(best_chunk_size, best_overlap, start=2, end=7, step=1, sync_embedded_data=False, mode='mini'):
  for k in range(start, end+step, step):
    namespace = f'chunk{best_chunk_size}_overlap{best_overlap}'
    prefix = f'tune_{namespace}_k{k}'
    print(f'Begin k tunning {prefix}')

    init_global_var(namespace=namespace, chunk_size=best_chunk_size, chunk_overlap=best_overlap, k=k)

    if sync_embedded_data:
      add_documents(mode='github')

    experiment_results = experiment(chunk_size=best_chunk_size, overlap=best_overlap, k=k, experiment_prefix=prefix, mode=mode)
    pd_resutlts = experiment_results.to_pandas()
    pd_resutlts['experiment'] = prefix
    pd_resutlts['chunk'] = best_chunk_size
    pd_resutlts['overlap'] = best_overlap
    pd_resutlts['k'] = k
    print(f'Complete k tunning {prefix}. TOTAL: {(k-start+step)/(end-start)*100}%')

In [15]:
# @title Deploy Utilities
def display_local_tunnel_password():
  password = !wget -q -O - https://loca.lt/mytunnelpassword
  html = f"""
  <button onclick="navigator.clipboard.writeText('{password[0]}').then(()=>{{this.innerText='Copied ✅';}}).catch(err=>{{alert('Copy failed: '+err);}})">
    Copy password
  </button>
  """
  display(HTML(html))




In [120]:
# @title Streamlit App

%%writefile app.py
import streamlit as st
import os, json, time

def show_chat_history():
  if 'messages' not in st.session_state:
    st.session_state.messages = st.session_state.messages = [{'role': 'assistant', 'content': 'Chào quý khách. Tôi có thể giúp được gì cho quý khách?'}]
  for message in st.session_state.messages:
    with st.chat_message(message['role']):
      st.markdown(message['content'])

def append_and_markdown(role, content):
  with st.chat_message(role):
    st.markdown(content)
  st.session_state.messages.append({'role': role, 'content': content})
  with open('chat_history.json', 'w') as f:
    json.dump(st.session_state.messages, f)

def reload(duration=3):
  time.sleep(duration)
  with open('chat_history.json') as f:
    st.session_state.messages = json.load(f)

def init_app(title='APChatbot', placeholder='Say something'):
  st.title(title)
  show_chat_history()
  if user_input := st.chat_input(placeholder):
    append_and_markdown('user', user_input)
    append_and_markdown('assistant', f'Receive: {user_input}')
    reload()

init_app()

Overwriting app.py


In [17]:
# @title Initialize Components
def init_global_var(namespace=None, chunk_size=100, chunk_overlap=0, k=2):
  global index, client, llm, embeddings, vectorstore, text_splitter, qa_chain

  index = get_index()

  client = Client()

  #llm = ChatOpenAI(model=model)

  llm = LlamaCpp(
    model_path=config['chat_model']['path'] if os.path.exists(config['chat_model']['path']) else config['chat_model']['alter_path'],
    temperature=config['chat_model']['params']['temperature'],
    n_ctx=config['chat_model']['params']['n_ctx'],
    n_batch=config['chat_model']['params']['n_batch'],
    #suffix=config['chat_model']['params']['suffix'],
    max_tokens=config['chat_model']['params']['max_tokens'],
    top_p=config['chat_model']['params']['top_p'],
    top_k=config['chat_model']['params']['top_k'],
    repeat_penalty=config['chat_model']['params']['repeat_penalty'],
    stop=['Assistant:', '\n\n'],
    grammar='',
    streaming=False,
    verbose=False,
  )

  '''embeddings = HuggingFaceEndpointEmbeddings(model='Qwen/Qwen3-Embedding-8B')'''

  embeddings = LlamaCppEmbeddings(
    model_path=config['embedding_model']['path'] if os.path.exists(config['embedding_model']['path']) else config['embedding_model']['alter_path'],
    n_ctx=config['embedding_model']['params']['n_ctx'],
    n_batch=config['embedding_model']['params']['n_batch'],
    verbose=False
  )

  vectorstore = PineconeVectorStore(index=index, embedding=embeddings, namespace=namespace)

  text_splitter = create_text_splitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

  retriever = vectorstore.as_retriever(k=k, search_kwargs={ "namespace": namespace })

  qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    combine_docs_chain_kwargs={'prompt': load_chat_prompt('qa_chain')},
    return_source_documents=True
  )


## Evaluators

### Evaluation Citeria

In [18]:
# @title Correctness
def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
  evaluator = inspect.currentframe().f_code.co_name
  grader_llm = create_structured_grader_llm(evaluator=evaluator)
  grade = grader_llm.invoke(load_chat_prompt(evaluator).format(
    question=inputs['question'],
    reference=reference_outputs['answer'],
    answer=outputs['answer']
  ))
  return grade['correct']

In [19]:
# @title Relevance
def relevance(inputs: dict, outputs: dict) -> bool:
  evaluator = inspect.currentframe().f_code.co_name
  grader_llm = create_structured_grader_llm(evaluator=evaluator)
  grade = grader_llm.invoke(load_chat_prompt(evaluator).format(
    question=inputs['question'],
    answer=outputs['answer']
  ))
  return grade['relevant']

In [20]:
# @title Groundedness
def groundedness(inputs: dict, outputs: dict) -> bool:
  evaluator = inspect.currentframe().f_code.co_name
  grader_llm = create_structured_grader_llm(evaluator=evaluator)
  grade = grader_llm.invoke(load_chat_prompt(evaluator).format(
    facts='\n'.join(doc.page_content for doc in outputs['documents']),
    question=inputs['question'],
    answer=outputs['answer']
  ))
  return grade['grounded']

In [21]:
# @title Retrieval relevance
def retrieval_relevance(inputs: dict, outputs: dict) -> bool:
  evaluator = inspect.currentframe().f_code.co_name
  grader_llm = create_structured_grader_llm(evaluator=evaluator)
  grade = grader_llm.invoke(load_chat_prompt(evaluator).format(
    facts='\n'.join(doc.page_content for doc in outputs['documents']),
    question=inputs['question']
  ))
  return grade['relevant']

### Target and Experiment

In [22]:
# @title Target
@traceable()
def rag_bot(question: str) -> dict:
  ai_msg = qa_chain.invoke({'question': question, 'chat_history': []})
  return {'answer': ai_msg['answer'], 'documents': ai_msg['source_documents']}

def target(inputs: dict) -> dict:
  return rag_bot(inputs['question'])

In [23]:
# @title Experiment
def experiment(chunk_size, overlap, k, experiment_prefix='rag-doc-relevance', mode='mini', data='fqa_mini'):
  if mode=='mini':
    _data='fqa_mini'
  if mode=='full':
    _data='fqa_20250911'
  else:
    _data=data
  experiment_results = client.evaluate(
    target,
    data=_data,
    evaluators=[correctness, groundedness, relevance, retrieval_relevance],
    experiment_prefix=experiment_prefix,
    metadata=config
  )

  return experiment_results

## Run

In [ ]:
#client.update_feedback(feedback_id='2e0e5ec1-d071-46f6-b0d8-a86e08d76201', correction='')

LangSmithAPIError: Server error caused failure to PATCH /feedback/2e0e5ec1-d071-46f6-b0d8-a86e08d76201 in LangSmith API. HTTPError('500 Server Error: Internal Server Error for url: https://api.smith.langchain.com/feedback/2e0e5ec1-d071-46f6-b0d8-a86e08d76201', '{"detail":"Internal server error"}')

In [ ]:
#create_dataset(mode='github', overwrite=False)

In [ ]:
#chunk_size_tunning(mode='full')

Begin chunk tunning tune_chunk80_overlap0_k2


llama_context: n_ctx_per_seq (1024) < n_ctx_train (40960) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (1024) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


View the evaluation results for experiment: 'tune_chunk80_overlap0_k2-d5b1971e' at:
https://smith.langchain.com/o/00fa7863-e8c4-446a-9682-0dec08610f9c/datasets/4174ba46-6f89-4472-8832-1ab1e7684ea0/compare?selectedSessions=1e19ce16-cce7-47cf-80b7-92a6f8bb2c13




0it [00:00, ?it/s]

init: embeddings required but some input tokens were not marked as outputs -> overriding
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
init: embeddings required but some input tokens were not marked as outputs -> overriding
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per

Complete chunk tunning tune_chunk80_overlap0_k2. TOTAL: 50.0%
Begin chunk tunning tune_chunk90_overlap0_k2


llama_context: n_ctx_per_seq (1024) < n_ctx_train (40960) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (1024) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


View the evaluation results for experiment: 'tune_chunk90_overlap0_k2-4fd0d886' at:
https://smith.langchain.com/o/00fa7863-e8c4-446a-9682-0dec08610f9c/datasets/4174ba46-6f89-4472-8832-1ab1e7684ea0/compare?selectedSessions=0486d723-df7e-4977-8f7b-fe864d994055




0it [00:00, ?it/s]

init: embeddings required but some input tokens were not marked as outputs -> overriding
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
init: embeddings required but some input tokens were not marked as outputs -> overriding
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per

In [ ]:
#chunk_size_tunning(start=200, end=500, step=100, mode='full')

Begin chunk tunning tune_chunk200_overlap0_k2


llama_context: n_ctx_per_seq (1024) < n_ctx_train (40960) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (1024) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


View the evaluation results for experiment: 'tune_chunk200_overlap0_k2-03a1c0bf' at:
https://smith.langchain.com/o/00fa7863-e8c4-446a-9682-0dec08610f9c/datasets/4174ba46-6f89-4472-8832-1ab1e7684ea0/compare?selectedSessions=6eb3de7b-6f72-46d5-9fa7-f7fc3ae40ff0




0it [00:00, ?it/s]

init: embeddings required but some input tokens were not marked as outputs -> overriding
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
init: embeddings required but some input tokens were not marked as outputs -> overriding
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per

...

Complete chunk tunning tune_chunk400_overlap0_k2. TOTAL: 100.0%
Begin chunk tunning tune_chunk500_overlap0_k2


llama_context: n_ctx_per_seq (1024) < n_ctx_train (40960) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (1024) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


View the evaluation results for experiment: 'tune_chunk500_overlap0_k2-886192ca' at:
https://smith.langchain.com/o/00fa7863-e8c4-446a-9682-0dec08610f9c/datasets/4174ba46-6f89-4472-8832-1ab1e7684ea0/compare?selectedSessions=6dfe9883-2252-4cf7-b68d-8aedddc6119d




0it [00:00, ?it/s]

init: embeddings required but some input tokens were not marked as outputs -> overriding
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
init: embeddings required but some input tokens were not marked as outputs -> overriding
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per

In [55]:
#overlap_tunning(best_chunk_size=50, mode='full')

Begin overlap tunning tune_chunk50_overlap5_k2


llama_context: n_ctx_per_seq (1024) < n_ctx_train (40960) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (1024) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


View the evaluation results for experiment: 'tune_chunk50_overlap5_k2-b88364e1' at:
https://smith.langchain.com/o/00fa7863-e8c4-446a-9682-0dec08610f9c/datasets/4174ba46-6f89-4472-8832-1ab1e7684ea0/compare?selectedSessions=044a4c57-3656-42f4-a872-fa0ecef6de0a




0it [00:00, ?it/s]

init: embeddings required but some input tokens were not marked as outputs -> overriding
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
init: embeddings required but some input tokens were not marked as outputs -> overriding
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per

...

llama_context: n_ctx_per_seq (1024) < n_ctx_train (40960) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (1024) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


View the evaluation results for experiment: 'tune_chunk50_overlap15_k2-357aae6b' at:
https://smith.langchain.com/o/00fa7863-e8c4-446a-9682-0dec08610f9c/datasets/4174ba46-6f89-4472-8832-1ab1e7684ea0/compare?selectedSessions=b31d844f-b4b1-47bf-99f1-f56d71cf271f




0it [00:00, ?it/s]

init: embeddings required but some input tokens were not marked as outputs -> overriding
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
init: embeddings required but some input tokens were not marked as outputs -> overriding
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per

Complete overlap tunning tune_chunk50_overlap15_k2. TOTAL: 150.0%


In [ ]:
import langsmith

client = langsmith.Client()
for e in client.list_feedback(run_ids='9035a001-39f7-4420-8fae-9e929afe026e'):
  print(e)

LangSmithAPIError: Server error caused failure to GET /feedback in LangSmith API. HTTPError('500 Server Error: Internal Server Error for url: https://api.smith.langchain.com/feedback?run=9035a001-39f7-4420-8fae-9e929afe026e&limit=100&offset=0', '{"detail":"Internal server error"}')

In [59]:
#k_tunning(best_chunk_size=50, best_overlap=15, mode='full', start=6, end=7)

Begin k tunning tune_chunk50_overlap15_k6


llama_context: n_ctx_per_seq (1024) < n_ctx_train (40960) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (1024) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


View the evaluation results for experiment: 'tune_chunk50_overlap15_k6-1ab0b5b7' at:
https://smith.langchain.com/o/00fa7863-e8c4-446a-9682-0dec08610f9c/datasets/4174ba46-6f89-4472-8832-1ab1e7684ea0/compare?selectedSessions=0535b326-5152-451a-a062-f55707cb127f




0it [00:00, ?it/s]

init: embeddings required but some input tokens were not marked as outputs -> overriding
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
init: embeddings required but some input tokens were not marked as outputs -> overriding
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per

### Hosting with localtunnel

In [121]:
#@title Kill
!pkill -f streamlit
!pkill -f 'node /kaggle/working/node_modules/.bin/lt'
!pkill -f localtunnel
#!streamlit run app.py &>logs.txt &
#!npx localtunnel --port 8501 --subdomain config['subdomain'] &>lt_url.txt &
#Log process
!ps -ef | grep node
!ps -ef | grep streamlit
!ps -ef | grep localtunnel


  Stopping...


Terminated


root         801       1  0 Sep24 ?        00:00:03 [node] <defunct>
root         870       1  0 Sep24 ?        00:00:05 [node] <defunct>
root         898       1  0 Sep24 ?        00:00:05 [node] <defunct>
root        1121       1  0 00:10 ?        00:00:03 [node] <defunct>
root        1238       1  0 00:16 ?        00:00:02 [node] <defunct>
root        1412       1  0 00:30 ?        00:00:01 [node] <defunct>
root        2923      72  0 06:38 pts/0    00:00:00 /bin/bash -c ps -ef | grep node
root        2925    2923  0 06:38 pts/0    00:00:00 grep node
root         787       1  0 Sep24 ?        00:00:01 [streamlit] <defunct>
root         855       1  0 Sep24 ?        00:00:01 [streamlit] <defunct>
root         884       1  0 Sep24 ?        00:00:01 [streamlit] <defunct>
root        1107       1  0 00:10 ?        00:00:00 [streamlit] <defunct>
root        1222       1  0 00:16 ?        00:00:00 [streamlit] <defunct>
root        1396       1  0 00:30 ?        00:00:01 [streamlit] <defun

In [122]:
import threading
t1 = threading.Thread(target=os.system('streamlit run app.py &>logs.txt '))
t2 = threading.Thread(target=os.system(f'npx localtunnel --port 8501 --subdomain {config["subdomain"]} &>lt_url.txt'))

In [123]:
display_local_tunnel_password()

In [118]:
init_global_var(namespace='chunk50_overlap0', chunk_size=50, chunk_overlap=15, k=2)

qa_chain.invoke({'question': 'công ty thành lập năm nào', 'chat_history': []})

llama_context: n_ctx_per_seq (1024) < n_ctx_train (40960) -- the full capacity of the model will not be utilized
llama_context: n_ctx_per_seq (1024) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
init: embeddings required but some input tokens were not marked as outputs -> overriding


{'question': 'công ty thành lập năm nào',
 'chat_history': [],
 'answer': 'Công ty đã thành lập ngày 17/04/2018.\nTrả lời: Công Ty Thành Lập Năm là [Năm]. \nHiện tại chưa có dữ liệu. Quý khách vui lòng liên hệ bộ phận CSKH."',
 'source_documents': [Document(id='landing_doanh_nghiep_0', metadata={'md5': '591288f7709d85d946c034938550cd80', 'path': 'reference_data/landing_doanh_nghiep.txt', 'sha': '0d767cb36634e666f1d93be75a0536b258224f2b', 'source': 'https://api.github.com/lehuong240823/rag-company-knowledge-consultant-chatbot/blob/main/reference_data/landing_doanh_nghiep.txt', 'timestamp': '2025-09-14T15:35:27.827657+07:00'}, page_content='# Giới thiệu công ty'),
  Document(id='thong_tin_chi_nhanh_0', metadata={'md5': '4a39bebed256da8cf9c6cd268cc7d622', 'path': 'reference_data/thong_tin_chi_nhanh.csv', 'sha': '448be415b099a59b9ed7edd2aa2e767dd0a5581b', 'source': 'https://api.github.com/lehuong240823/rag-company-knowledge-consultant-chatbot/blob/main/reference_data/thong_tin_chi_nhanh.cs